In [ ]:
#here we use the model we create for prediction
import joblib
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Generate random training data (1000 samples, 448 features)
X_train = np.random.uniform(0.25, 0.75, (1000, 448))

# Step 1: Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

# Step 2: Apply PCA (reduce 448 → 3 components)
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# Save the trained models
joblib.dump(scaler, "scaler.pkl")
joblib.dump(pca, "pca.pkl")

print("✅ Scaler and PCA saved successfully!")
import numpy as np
import joblib

# Load the saved scaler and PCA
scaler = joblib.load("scaler.pkl")
pca = joblib.load("pca.pkl")

# Create a new input feature vector (1 sample, 448 features)
input_features = np.random.uniform(0.25, 0.75, (1, 448))  # Shape: (1, 448)

# Standardize the input
input_scaled = scaler.transform(input_features)

# Apply PCA transformation (reduces to 3 features)
input_pca = pca.transform(input_scaled)

# Reshape for feeding into LSTM (if required)
input_final = input_pca.reshape((input_pca.shape[0], 1, input_pca.shape[1]))

print("✅ Input processed: Ready for model prediction!")
print("Processed shape:", input_final.shape)  # Should be (1, 1, 3)


✅ Scaler and PCA saved successfully!
✅ Input processed: Ready for model prediction!
Processed shape: (1, 1, 3)


In [ ]:
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras.losses import MeanSquaredError

# Load the trained LSTM model
custom_objects = {"mse": MeanSquaredError()}
model = tf.keras.models.load_model("/content/best_lstm_model.h5", custom_objects=custom_objects)
print("Model loaded successfully!")

# Load the Standard Scaler (for input) & PCA
scaler = joblib.load("/content/scaler.pkl")  # StandardScaler for input features
pca = joblib.load("/content/pca.pkl")  # PCA for input dimensionality reduction

# Load the Output Scaler (for target variable) which was used during prediction
output_scaler = joblib.load("/content/output_scaler.pkl")  # StandardScaler for model output

#  Generate Random Input (1, 448)
input_features = np.random.uniform(0.25, 0.75, (1, 448))

# Standardize & Apply PCA
input_scaled = scaler.transform(input_features)  # Shape: (1, 448)
input_pca = pca.transform(input_scaled)  # Shape: (1, 3)

# Reshape for LSTM (Batch=1, Timesteps=1, Features=3)
input_final = input_pca.reshape((1, 1, 3))

#  Predict using LSTM
prediction_scaled = model.predict(input_final)  # LSTM output (in scaled space)

#  Reverse Scaling using the target scaler (Not PCA)
final_output = output_scaler.inverse_transform(prediction_scaled)  # Convert back to original target scale

#  Display Results
print(" Prediction completed!")
print("Final Output Shape:", final_output.shape)  # Should be (1, X) where X matches target shape
print("Final Output Values:", final_output)


✅ Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 572ms/step
✅ Prediction completed!
Final Output Shape: (1, 1)
Final Output Values: [[5.1169143]]
